# Compare different similarity methods

- Pocket structure similarity (`kissim`)
- Pocket sequence identity (KLIFS seq)
- Interaction similarity (KLIFS IFP)
- Compare matrices!

Resources:

- [SciPy: Jaccard distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jaccard.html#scipy.spatial.distance.jaccard)
- [Scikit-Learn: Pairwise distances](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import scipy
import seaborn as sns
from opencadd.databases import klifs

import src
from src.paths import PATH_DATA, PATH_RESULTS

In [3]:
HERE = Path(_dh[-1])  # noqa: F821
DATA = PATH_DATA
RESULTS = PATH_RESULTS

In [4]:
klifs_session = klifs.setup_remote()

In [5]:
def min_max_normalize_df(df):
    df = (df - df.min().min()) / (df.max().max() - df.min().min())
    return df

## Pocket structure similarity: `kissim`

### Kinase matrix

In [6]:
%%time
kissim_matrix = src.data.distances.kissim()

CPU times: user 20 s, sys: 1.66 s, total: 21.7 s
Wall time: 21.6 s


### Get kinases and structure KLIFS IDs from `kissim` dataset

#### Kinase names

In [7]:
kinase_names = kissim_matrix.columns.to_list()
print(f"Number of kinase names: {len(kinase_names)}")

Number of kinase names: 257


#### Kinase KLIFS IDs

In [8]:
kinases = klifs_session.kinases.by_kinase_name(kinase_names, species="Human")
# Make sure the DataFrame is ordered by kinase names in list
kinases = kinases.set_index("kinase.klifs_name").loc[kinase_names].reset_index()
kinase_klifs_ids = kinases["kinase.klifs_id"].to_list()
print(f"Number of kinase KLIFS IDs: {len(kinase_klifs_ids)}")

Number of kinase KLIFS IDs: 257


#### Structure KLIFS IDs

In [9]:
structures = klifs_session.structures.by_kinase_klifs_id(kinase_klifs_ids)
structure_klifs_ids = structures["structure.klifs_id"].to_list()
print(f"Number of structure KLIFS IDs: {len(structure_klifs_ids)}")

Number of structure KLIFS IDs: 11468


## Pocket sequence identity: KLIFS sequence

Settings:

- Keep full pocket sequences only (can contain gaps)
- Sequence identity!
- Matrix values: distances (not similarities)
- Min-max normalization

### Kinase matrix

In [10]:
%%time
seq_matrix = src.data.distances.klifs_pocket_sequence()

CPU times: user 374 ms, sys: 4.1 ms, total: 378 ms
Wall time: 519 ms


## Interaction similarity: KLIFS IFP

Settings:

- Use structures in DFG-in only
- Use Jaccard distance
- Matrix values: distances (not similarities)
- Min-max normalization

### Kinase matrix

In [11]:
%%time
ifp_matrix = src.data.distances.klifs_pocket_ifp(structure_klifs_ids)

KeyError: 'interaction.fingerprint'

## Compare matrices

### Shared kinases

In [12]:
kissim_matrix.shape, seq_matrix.shape, ifp_matrix.shape

NameError: name 'ifp_matrix' is not defined

In [13]:
kinase_names_selected = set(kissim_matrix.columns.to_list()) & set(seq_matrix.columns.to_list())
kinase_names_selected = kinase_names_selected & set(ifp_matrix.columns.to_list())
kinase_names_selected = sorted(kinase_names_selected)
print(f"Number of shared kinases: {len(kinase_names_selected)}")

NameError: name 'ifp_matrix' is not defined

### Shared kinase distance matrix

In [14]:
kissim_matrix_shared = kissim_matrix.loc[kinase_names_selected, kinase_names_selected]
seq_matrix_shared = seq_matrix.loc[kinase_names_selected, kinase_names_selected]
ifp_matrix_shared = ifp_matrix.loc[kinase_names_selected, kinase_names_selected]

print("KiSSim", kissim_matrix_shared.min().min(), kissim_matrix_shared.max().max())
print("Seq   ", seq_matrix_shared.min().min(), seq_matrix_shared.max().max())
print("IFP   ", ifp_matrix_shared.min().min(), ifp_matrix_shared.max().max())

NameError: name 'ifp_matrix' is not defined

### Shared normalized kinase distance matrix

In [15]:
kissim_matrix_shared_normalized = min_max_normalize_df(kissim_matrix_shared)
seq_matrix_shared_normalized = min_max_normalize_df(seq_matrix_shared)
ifp_matrix_shared_normalized = min_max_normalize_df(ifp_matrix_shared)

print(
    "KiSSim",
    kissim_matrix_shared_normalized.min().min(),
    kissim_matrix_shared_normalized.max().max(),
)
print("Seq   ", seq_matrix_shared_normalized.min().min(), seq_matrix_shared_normalized.max().max())
print("IFP   ", ifp_matrix_shared_normalized.min().min(), ifp_matrix_shared_normalized.max().max())

NameError: name 'ifp_matrix_shared' is not defined

### Shared kinase pair distances

In [16]:
kissim_pairs = kissim_matrix_shared_normalized.to_numpy()[
    np.tril_indices(kissim_matrix_shared_normalized.to_numpy().shape[0], -1)
]
seq_pairs = seq_matrix_shared_normalized.to_numpy()[
    np.tril_indices(seq_matrix_shared_normalized.to_numpy().shape[0], -1)
]
ifp_pairs = ifp_matrix_shared_normalized.to_numpy()[
    np.tril_indices(ifp_matrix_shared_normalized.to_numpy().shape[0], -1)
]

pair_distances_df = pd.DataFrame(
    {"KiSSim": kissim_pairs, "KLIFS pocket sequence": seq_pairs, "KLIFS pocket IFP": ifp_pairs}
)
pair_distances_df.head()

NameError: name 'ifp_matrix_shared_normalized' is not defined

### Plot distributions

- https://seaborn.pydata.org/tutorial/distributions.html
- https://seaborn.pydata.org/generated/seaborn.pairplot.html

In [17]:
def annotate(x, y, ax):
    """
    Annotate axes with correlation coefficient.
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html
    """
    r, p = scipy.stats.pearsonr(x, y)
    ax.text(0.05, 0.95, f"r={round(r, 2)}", transform=ax.transAxes)

In [18]:
g = sns.PairGrid(pair_distances_df)
# Turn off grid
for axes in g.axes:
    for ax in axes:
        ax.grid(False)

g.map_upper(sns.regplot, scatter_kws={"s": 1}, ci=95)
g.map_lower(sns.kdeplot, fill=True)
g.map_diag(sns.histplot, kde=True)

# Annotate
tmp = pair_distances_df[["KiSSim", "KLIFS pocket sequence"]].dropna()
annotate(tmp["KiSSim"], tmp["KLIFS pocket sequence"], g.axes[0][1])
tmp = pair_distances_df[["KiSSim", "KLIFS pocket IFP"]].dropna()
annotate(tmp["KiSSim"], tmp["KLIFS pocket IFP"], g.axes[0][2])
tmp = pair_distances_df[["KLIFS pocket sequence", "KLIFS pocket IFP"]].dropna()
annotate(tmp["KLIFS pocket sequence"], tmp["KLIFS pocket IFP"], g.axes[1][2])

g.savefig(HERE / "../../notebooks/figures/figX_method_comparison_pairplots.pdf")

NameError: name 'pair_distances_df' is not defined